In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df = pd.read_csv('train_cleaned.csv')

In [4]:
df.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Item_Fat_Content_LF,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,...,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,1999,3735.1380,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,0
1,5.92,0.019278,48.2692,2009,443.4228,0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
2,17.50,0.016760,141.6180,1999,2097.2700,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,0
3,19.20,0.000000,182.0950,1998,732.3800,0,0,1,0,0,...,0,0,0,0,0,1,1,0,0,0
4,8.93,0.000000,53.8614,1987,994.7052,0,1,0,0,0,...,1,0,0,0,0,1,0,1,0,0


In [5]:
x=df.drop(['Item_Outlet_Sales'],axis=1)
y=df['Item_Outlet_Sales']
y.shape

(8523,)

In [7]:
from sklearn.model_selection import train_test_split as tts
train_x,valid_x,train_y,valid_y=tts(x,y,random_state=101,shuffle=False)

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor


In [12]:
model1=LinearRegression()
model1.fit(train_x,train_y)
pred1=model1.predict(valid_x)
model1.score(valid_x,valid_y)

(0.5551582433748833, 0.5658331947629718)

In [13]:
model2=KNeighborsRegressor(n_neighbors=9)
model2.fit(train_x,train_y)
pred2=model2.predict(valid_x)
model2.score(valid_x,valid_y)

0.5007936087911619

In [14]:
model3=DecisionTreeRegressor(max_depth=7)
model3.fit(train_x,train_y)
pred3=model3.predict(valid_x)
model3.score(valid_x,valid_y)

0.5735111622038214

In [36]:
from statistics import mean
final_pred=np.array([])
for i in range(0,len(valid_x)):
    final_pred=np.append(final_pred,mean([pred1[i],pred2[i],pred3[i]]))

In [17]:
m1_score=model1.score(valid_x,valid_y)
m2_score=model2.score(valid_x,valid_y)
m3_score=model3.score(valid_x,valid_y)

In [22]:
valid_r2=[m1_score,m2_score,m3_score]
index_=[1,2,3]
rank_eval=pd.DataFrame({'score':valid_r2},index=index_)

In [24]:
sorted_rank=rank_eval.sort_values('score')
sorted_rank

,score
2,0.500794
1,0.555158
3,0.573511


In [25]:
sorted_rank['rank']=[i for i in range(1,4)]
sorted_rank

,score,rank
2,0.500794,1
1,0.555158,2
3,0.573511,3


In [26]:
sorted_rank['weight']=sorted_rank['rank']/sorted_rank['rank'].sum()
sorted_rank

,score,rank,weight
2,0.500794,1,0.166667
1,0.555158,2,0.333333
3,0.573511,3,0.500000


In [32]:

wt_pred1=pred1*float(sorted_rank.loc[[1],['weight']].values)
wt_pred2=pred2*float(sorted_rank.loc[[2],['weight']].values)

In [33]:
wt_pred3=pred3*float(sorted_rank.loc[[3],['weight']].values)

In [34]:
ranked_pred=wt_pred1+wt_pred2+wt_pred3
ranked_pred

array([2331.19418533, 2760.66363737, 1452.6721998 , ..., 1348.78554185,
       1570.52823782, 1268.26795746])

In [38]:
from sklearn.metrics import r2_score
r2_score(valid_y,ranked_pred)


0.5828316440450751